<a href="https://colab.research.google.com/github/spongebob03/Webtoon-Recommendation/blob/NLP/Ntoon_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텍스트 유사도

## Import module

In [1]:
from datetime import timedelta, datetime
import json
import os
import re

import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from pandas.plotting import register_matplotlib_converters
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Fetched 252 kB in 2s (107 kB/s)
Reading package lis

In [3]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


## 웹툰 텍스트 유사도
참고 블로그
  - https://code-ing.tistory.com/2

### 데이터 준비

In [4]:
from google.colab import drive

data = pd.read_csv('/content/drive/MyDrive/n-simple.csv')
detail = pd.read_csv('/content/drive/MyDrive/n-detail.csv')

df = pd.merge(data, detail, how='left', on='id')
df.rename(columns={"title_x":"title"}, inplace=True)
df.drop(['title_y', 'link'], axis=1, inplace=True)

# 장르 -> 진행유형, 장르 분리
df['genre_list'] = df.genre.str.split(', ')
df.drop(['genre'], axis=1, inplace=True)
df[['gnr1', 'gnr2']] = pd.DataFrame(df.genre_list.tolist(), index=df.index)
df.drop(['genre_list'], axis=1, inplace=True)

df.head()

,id,title,rating,completed,date,author,description,age,free,thumbnail,gnr1,gnr2
0,756056,가난을 등에 업은 소녀,9.11,True,2020.10.19 00:00,B급달궁 / 오은지,흔하디 흔한 재벌후계자와 캔디도 울고 갈 박복한 가난소녀의 파란만장 동거기!인기작 ...,전체연령가,True,https://shared-comic.pstatic.net/thumb/webtoon...,스토리,로맨스
1,670144,가담항설,9.98,True,2020.10.07 23:00,랑또,이번 주인공은 돌이다!돌이지만 동료도 모으고 악당도 물리친다!랑또 작가표 동양 판타...,12세 이용가,True,https://shared-comic.pstatic.net/thumb/webtoon...,스토리,판타지
2,732071,가령의 정체불명 이야기,9.95,True,2020.02.03 23:22,가령,어느 날... 인어가 내게 말을 걸어왔다. 눈을 떠보니 총구가 있었다. 그리고 몸이...,15세 이용가,True,https://shared-comic.pstatic.net/thumb/webtoon...,옴니버스,드라마
3,703844,가비지타임,9.96,False,2022.04.23 22:57,2사장,"한국 최초 한국형 고교스포츠 웹툰!열혈따윈 개나 줘, 낙오자들 뿐인 농구부의 운명은?",전체연령가,False,https://shared-comic.pstatic.net/thumb/webtoon...,스토리,스포츠
4,785701,가상&RPG,9.94,False,2022.04.20 22:56,주다현,"""사람은 좀 죽여도 싸우는건 싫어해요."" 게임에서조차 집 바깥을 나가지 않는 '제리...",12세 이용가,False,https://shared-comic.pstatic.net/thumb/webtoon...,스토리,판타지


In [5]:
all_gnr = set(df['gnr2'])
all_gnr

{'감성', '개그', '드라마', '로맨스', '무협/사극', '스릴러', '스포츠', '액션', '일상', '판타지'}

### 데이터 전처리

In [6]:
from konlpy.tag import Kkma, Hannanum, Komoran
from konlpy.utils import pprint

hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()

#웹툰 설명에 대한 stopword 설정
stop_words = set("를 의 웹 툰 웹툰 웹툰판 이번 들 등 수  이 부 판 뿐 여자 남자 그 것 나 그 그녀 속 시작 속".split())
stop_words |= set("이야기 데 전 후 두 앞 뒤 그들 때문 사람 두 신작 한 자신 소년 소녀 만화".split())

In [7]:
def get_voca(doclist):
    doc_nouns_list = []

    for doc in doclist:
        nouns = hannanum.nouns(doc)
        doc_nouns = ''

        for noun in nouns:
            if noun not in stop_words:
                doc_nouns += noun + ' '

        doc_nouns_list.append(doc_nouns)
    return doc_nouns_list

### TF-IDF

In [8]:
def recomend_by_word(doclist, result):
    # 결과 웹툰 식별을 위한 딕셔너리
    id_dict = {i: id for i, id in enumerate(doclist['id'])}

    descriptions = doclist['description']
    voca = get_voca(descriptions)

    # TF-IDF
    tfidf_vectorizer = TfidfVectorizer(min_df=1)
    tfidf_matrix = tfidf_vectorizer.fit_transform(voca)
    document_distances = tfidf_matrix * tfidf_matrix.T

    print('유사도 분석을 위한 ' + str(document_distances.get_shape()[0]) + 'x' + str(document_distances.get_shape()[1]) + 'matrix를 만들었습니다.', end='->')
    n = document_distances.get_shape()[0]

    cnt = 0
    for i in range(n):
        a = document_distances[i].toarray().T
        similarity = set()
        for j in range(n):
            if a[j][0] > 0.5 and j != i: # 유사도 0.5 이상인 다른 웹툰
                similarity.add(id_dict[j]) # 웹툰번호로 저장
        if len(similarity) > 0:
            result[id_dict[i]] = result.get(id_dict[i], set([])) | set(similarity)
            cnt += 1

    print(f'{cnt}개 웹툰에 대한 추천')
    return result

전체 웹툰에서의 키워드 유사도 추천

In [9]:
result = recomend_by_word(df[['id', 'description']], {})

유사도 분석을 위한 1791x1791matrix를 만들었습니다.->104개 웹툰에 대한 추천


In [10]:
for gnr_name in all_gnr:
    print(f'in {gnr_name}...', end=' ')
    doclist = df[df['gnr2']==gnr_name][['id', 'description']]
    recomend_by_word(doclist, result)

in 판타지... 유사도 분석을 위한 323x323matrix를 만들었습니다.->7개 웹툰에 대한 추천
in 스릴러... 유사도 분석을 위한 220x220matrix를 만들었습니다.->12개 웹툰에 대한 추천
in 일상... 유사도 분석을 위한 75x75matrix를 만들었습니다.->0개 웹툰에 대한 추천
in 무협/사극... 유사도 분석을 위한 26x26matrix를 만들었습니다.->2개 웹툰에 대한 추천
in 액션... 유사도 분석을 위한 147x147matrix를 만들었습니다.->2개 웹툰에 대한 추천
in 로맨스... 유사도 분석을 위한 404x404matrix를 만들었습니다.->11개 웹툰에 대한 추천
in 개그... 유사도 분석을 위한 133x133matrix를 만들었습니다.->10개 웹툰에 대한 추천
in 감성... 유사도 분석을 위한 33x33matrix를 만들었습니다.->0개 웹툰에 대한 추천
in 스포츠... 유사도 분석을 위한 23x23matrix를 만들었습니다.->0개 웹툰에 대한 추천
in 드라마... 유사도 분석을 위한 407x407matrix를 만들었습니다.->19개 웹툰에 대한 추천


In [11]:
print(len(result))

106


In [12]:
# 웹툰 번호 제목으로 표기해서 출력
def show_result(result):
    i = 1
    for key, value in result.items():
        print(i, df[df['id']==key].title.item(), end='\n👉👉 ')
        print(' / '.join([df[df['id']==v].title.item() for v in value]))
        print('---'*15)
        i += 1

In [13]:
show_result(result)

1 가우스전자 시즌1~2
👉👉 가우스전자 시즌3~4
---------------------------------------------
2 가우스전자 시즌3~4
👉👉 가우스전자 시즌1~2
---------------------------------------------
3 간질간질
👉👉 꽃 피는 날 / 안녕, 대학생 / 수학 잘하는 법
---------------------------------------------
4 감염자
👉👉 시노딕
---------------------------------------------
5 개미
👉👉 우바우
---------------------------------------------
6 공방의 마녀
👉👉 몸이 바뀌는 사정
---------------------------------------------
7 구석구석캠페인
👉👉 우리의 이상한 여행
---------------------------------------------
8 금요일 베스트
👉👉 원주민 공포만화
---------------------------------------------
9 꽃 피는 날
👉👉 안녕, 대학생 / 간질간질
---------------------------------------------
10 난약
👉👉 컨트롤제트
---------------------------------------------
11 너와 너 사이
👉👉 드래곤레시피
---------------------------------------------
12 놓지마 정신줄
👉👉 놓지마 정신줄 시즌2
---------------------------------------------
13 놓지마 정신줄 시즌2
👉👉 놓지마 정신줄
---------------------------------------------
14 단편 할머니/나는내일죽는다
👉👉 초록인간
---------------------------------------------
15 달콤한 인생
👉👉 달콤한 인생_스페셜 에피

In [14]:
def find_recommend(result, title):
    print(f'웹툰 [{title}]과 비슷한 다른 웹툰은...', end=' ')
    others = result.get(df[df['title']==title].id.item(), [])
    if not others:
        print('아직 없습니다.')
        return
    print(' / '.join([df[df['id']==s].title.item() for s in others]))

In [16]:
find_recommend(result, '해피')

웹툰 [해피]과 비슷한 다른 웹툰은... 지새는 달 / 영수의 봄 / 일사부재리 / 레드돌
